In [1]:
# --- Librerías ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy.stats import pointbiserialr
from sklearn.preprocessing import PowerTransformer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.base import BaseEstimator
import os

# Configuración de gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

In [2]:
import logging
import pandas as pd
from scipy.stats import pointbiserialr
from sklearn.preprocessing import PowerTransformer

# Configuración del logger
logger = logging.getLogger('preprocessor_logger')
logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - [%(funcName)s] - %(message)s'
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)


class Preprocessor:
    def __init__(self, df: pd.DataFrame, target_col: str = 'target'):
        """
        Inicializa el preprocesador con el DataFrame original.
        
        Args:
            df: DataFrame a procesar
            target_col: Nombre de la columna target
        """
        self.target_col = target_col
        self.original_df = df.copy()
        self.processed_df = None
        self.cor_target = None
        self.product_cols = None
        
        # Atributos para exportar
        self.zones_ = None
        self.zone_mapper_df = None
        self.cols_to_drop_ = []
        self.power_params_ = None
        self.skewed_cols_ = []
        self.high_corr_df_products = None
        
        logger.info(f"Preprocessor inicializado - Shape original: {self.original_df.shape}")

    def _log_shape_change(self, previous_shape: tuple, operation: str):
        """Registra cambios en el shape del DataFrame."""
        current_shape = self.processed_df.shape
        logger.info(
            f"{operation} - Shape cambiado de {previous_shape} a {current_shape} "
            f"(Filas: {previous_shape[0]} → {current_shape[0]}, "
            f"Columnas: {previous_shape[1]} → {current_shape[1]})"
        )

    def group_sociodemographic_cols(self, sociodemographic_cols: list):
        """
        Agrupa columnas sociodemográficas en una sola columna 'zone'.
        
        Args:
            sociodemographic_cols: Lista de columnas sociodemográficas a agrupar
        """
        logger.info(f"Iniciando agrupación de columnas sociodemográficas: {sociodemographic_cols}")
        
        previous_shape = self.original_df.shape
        
        # Crear columna zone basada en agrupación
        self.original_df['zone'] = self.original_df.groupby(sociodemographic_cols).ngroup() + 1
        
        # Crear mapper de zonas
        expanded_cols = sociodemographic_cols + ['zone']
        self.zone_mapper_df = self.original_df[expanded_cols].drop_duplicates()
        
        self.zones_ = [str(col) for col in self.zone_mapper_df['zone'].unique()]
        
        # Eliminar columnas originales
        self.original_df.drop(columns=sociodemographic_cols, inplace=True)
        self.processed_df = self.original_df.copy()
        
        logger.info(f"Agrupación completada - {len(self.zone_mapper_df)} zonas únicas creadas")
        self._log_shape_change(previous_shape, "Agrupación sociodemográfica")

    def remove_duplicates(self):
        """Elimina duplicados exactos del DataFrame."""
        logger.info("Iniciando eliminación de duplicados exactos")
        
        previous_shape = self.processed_df.shape
        initial_rows = len(self.processed_df)
        
        self.processed_df = self.processed_df.drop_duplicates()
        final_rows = len(self.processed_df)
        removed_rows = initial_rows - final_rows
        
        logger.info(f"Duplicados eliminados: {removed_rows} filas removidas")
        self._log_shape_change(previous_shape, "Eliminación de duplicados exactos")

    def handle_complex_duplicates(self):
        """
        Maneja duplicados complejos donde las filas son idénticas excepto por el target.
        Conserva la moda del target en casos de múltiples duplicados.
        """
        logger.info("Iniciando manejo de duplicados complejos")
        
        df = self.processed_df.copy()
        previous_shape = df.shape
        initial_rows = len(df)
        
        # Agrupar filas duplicadas excepto por target
        columns = list(set(df.columns) - {self.target_col})
        grupos_duplicados = df.groupby(columns).groups
        
        indices_a_eliminar = []
        grupos_procesados = 0
        
        for fila, indices in grupos_duplicados.items():
            if len(indices) > 1:  # Solo grupos con duplicados
                grupos_procesados += 1
                grupo_actual = df.loc[indices]
                
                if len(indices) == 2:
                    # Eliminar todo el grupo (ambas filas)
                    indices_a_eliminar.extend(indices)
                else:
                    # Grupos con más de 2 filas: conservar solo la moda
                    moda_target = grupo_actual[self.target_col].mode()
                    
                    if len(moda_target) > 0:
                        moda = moda_target[0]
                        # Conservar solo las filas con target = moda
                        filas_a_eliminar = grupo_actual[grupo_actual[self.target_col] != moda].index
                        indices_a_eliminar.extend(filas_a_eliminar)
                    else:
                        # Si no hay moda clara, eliminar todo el grupo
                        indices_a_eliminar.extend(indices)
        
        # Crear nuevo DataFrame sin los duplicados problemáticos
        self.processed_df = df.drop(indices_a_eliminar)
        final_rows = len(self.processed_df)
        removed_rows = initial_rows - final_rows
        
        logger.info(
            f"Manejo de duplicados complejos completado - "
            f"{grupos_procesados} grupos procesados, {removed_rows} filas eliminadas"
        )
        self._log_shape_change(previous_shape, "Manejo de duplicados complejos")

    def get_correlations(self):
        """Calcula correlaciones punto-biserial entre features y target."""
        logger.info("Calculando correlaciones con el target")
        
        if self.target_col not in self.processed_df.columns:
            raise ValueError(f"Target column '{self.target_col}' no encontrada en el DataFrame")
        
        cor_target = {}
        features = [col for col in self.processed_df.columns if col != self.target_col]

        for col in features:
            corr, _ = pointbiserialr(self.processed_df[col], self.processed_df[self.target_col])
            cor_target[col] = corr

        self.cor_target = pd.Series(cor_target).sort_values(key=abs, ascending=False)
        
        logger.info(
            f"Correlaciones calculadas - "
            f"Rango: [{self.cor_target.min():.3f}, {self.cor_target.max():.3f}], "
            f"Top 3: {self.cor_target.head(3).to_dict()}"
        )

    def get_high_pair_correlations(self, product_cols: list):
        """
        Identifica pares de variables con alta correlación entre sí.
        
        Args:
            product_cols: Lista de columnas de productos a analizar
        """
        logger.info(f"Buscando correlaciones altas entre {len(product_cols)} columnas de productos")
        
        # Verificar que las columnas existen
        missing_cols = set(product_cols) - set(self.processed_df.columns)
        if missing_cols:
            raise ValueError(f"Columnas no encontradas: {missing_cols}")
        
        corr_abs_products = self.processed_df[product_cols].corr().abs()

        high_corr_pairs_products = []
        for i in range(len(corr_abs_products.columns)):
            for j in range(i+1, len(corr_abs_products.columns)):
                correlation = corr_abs_products.iloc[i, j]
                if correlation > 0.7:
                    high_corr_pairs_products.append({
                        'var1': corr_abs_products.columns[i],
                        'var2': corr_abs_products.columns[j], 
                        'correlation': correlation
                    })

        self.high_corr_df_products = pd.DataFrame(high_corr_pairs_products).sort_values(
            'correlation', ascending=False
        )
        
        # Filtrar correlaciones muy altas
        high_corr_count = len(self.high_corr_df_products[self.high_corr_df_products['correlation'] > 0.95])
        self.product_cols = product_cols
        
        logger.info(
            f"Análisis de correlación completado - "
            f"{len(high_corr_pairs_products)} pares con correlación > 0.7, "
            f"{high_corr_count} pares con correlación > 0.95"
        )

    def drop_high_correlated_cols(self):
        """Elimina columnas altamente correlacionadas, conservando las más relevantes."""
        logger.info("Iniciando eliminación de columnas altamente correlacionadas")
        
        if self.high_corr_df_products is None:
            raise ValueError("Debe ejecutar get_high_pair_correlations primero")
        
        previous_shape = self.processed_df.shape
        initial_cols = len(self.processed_df.columns)
        
        high_corr_filtered = self.high_corr_df_products[self.high_corr_df_products['correlation'] > 0.95]
        
        for _, row in high_corr_filtered.iterrows():
            var1 = row['var1']
            var2 = row['var2']

            # Comparar la correlación absoluta con el target
            corr_var1 = abs(self.cor_target[var1])
            corr_var2 = abs(self.cor_target[var2])

            # Quedarse con la columna más correlacionada, eliminar la otra
            if corr_var1 < corr_var2:
                col_to_drop = var1
                col_to_keep = var2
            else:
                col_to_drop = var2
                col_to_keep = var1
            
            if col_to_drop not in self.cols_to_drop_:
                self.cols_to_drop_.append(col_to_drop)
                logger.debug(f"Marcada para eliminar: {col_to_drop} (corr: {corr_var1:.3f}) "
                           f"vs {col_to_keep} (corr: {corr_var2:.3f})")

        self.cols_to_drop_ = list(set(self.cols_to_drop_))
        self.product_cols = list(set(product_cols)-set(self.cols_to_drop_))
        
        # Eliminar columnas
        columns_before_drop = set(self.processed_df.columns)
        self.processed_df = self.processed_df.drop(columns=self.cols_to_drop_)
        columns_after_drop = set(self.processed_df.columns)
        dropped_columns = columns_before_drop - columns_after_drop
        
        final_cols = len(self.processed_df.columns)
        
        logger.info(
            f"Eliminación de columnas correlacionadas completada - "
            f"{len(dropped_columns)} columnas eliminadas: {list(dropped_columns)}"
        )
        self._log_shape_change(previous_shape, "Eliminación de columnas correlacionadas")

    def correct_skewness(self):
        """
        Corrige asimetría en las columnas usando transformación Yeo-Johnson.
        
        Args:
            product_cols: Lista de columnas de productos a transformar
        """
        logger.info("Iniciando corrección de asimetría")
        
        # Calcular asimetría inicial
        skewness_before = self.processed_df[self.product_cols].skew()
        self.skewed_cols_ = skewness_before[abs(skewness_before) > 0.5].index.tolist()
        
        if not self.skewed_cols_:
            logger.info("No se encontraron columnas con asimetría significativa (> 0.5)")
            return
            
        logger.info(f"{len(self.skewed_cols_)} columnas con asimetría > 0.5: {self.skewed_cols_}")
        
        # Aplicar transformación Yeo-Johnson
        pt = PowerTransformer(method='yeo-johnson')
        self.processed_df[self.skewed_cols_] = pt.fit_transform(self.processed_df[self.skewed_cols_])
        
        # Calcular asimetría después de la transformación
        skewness_after = self.processed_df[self.skewed_cols_].skew()
        
        # Guardar parámetros
        if hasattr(pt, "lambdas_"):
            self.power_params_ = dict(zip(self.skewed_cols_, pt.lambdas_))
        else:
            self.power_params_ = {}
        
        logger.info(
            f"Corrección de asimetría completada - "
            f"Asimetría promedio: {skewness_before.mean():.3f} → {skewness_after.mean():.3f}"
        )

    def apply_one_hot(self):
        """Aplica one-hot encoding a la columna 'zone'."""
        logger.info("Aplicando one-hot encoding a la columna 'zone'")
        
        if 'zone' not in self.processed_df.columns:
            raise ValueError("Columna 'zone' no encontrada para one-hot encoding")
        
        previous_shape = self.processed_df.shape
        
        # Convertir a entero y aplicar one-hot
        self.processed_df['zone'] = self.processed_df['zone'].astype(int)
        zone_dummies = pd.get_dummies(self.processed_df['zone'], prefix='zone')
        
        # Concatenar y eliminar columna original
        self.processed_df = pd.concat([
            self.processed_df.drop('zone', axis=1), 
            zone_dummies
        ], axis=1)
        
        logger.info(f"One-hot encoding completado - {len(zone_dummies.columns)} columnas zone creadas")
        self._log_shape_change(previous_shape, "One-hot encoding")

    def apply_preprocess(self, sociodemographic_cols: list, product_cols: list) -> pd.DataFrame:
        """
        Ejecuta el pipeline completo de preprocesamiento.
        
        Args:
            sociodemographic_cols: Columnas sociodemográficas a agrupar
            product_cols: Columnas de productos para análisis de correlación
            
        Returns:
            DataFrame procesado
        """
        logger.info("=== INICIANDO PIPELINE COMPLETO DE PREPROCESAMIENTO ===")
        logger.info(f"Columnas sociodemográficas: {sociodemographic_cols}")
        logger.info(f"Columnas de productos: {product_cols}")
        
        # Pipeline de procesamiento
        self.group_sociodemographic_cols(sociodemographic_cols)
        self.remove_duplicates()
        self.handle_complex_duplicates()
        self.get_correlations()
        self.get_high_pair_correlations(product_cols)
        self.drop_high_correlated_cols()
        self.correct_skewness()
        self.apply_one_hot()
        
        logger.info("=== PIPELINE COMPLETADO EXITOSAMENTE ===")
        logger.info(f"Shape final del DataFrame: {self.processed_df.shape}")
        logger.info(f"Columnas eliminadas: {len(self.cols_to_drop_)}")
        logger.info(f"Columnas transformadas: {len(self.skewed_cols_)}")
        
        return self.processed_df

    def get_preprocessing_summary(self) -> dict:
        """Retorna un resumen del proceso de preprocesamiento."""
        return {
            'original_shape': self.original_df.shape,
            'processed_shape': self.processed_df.shape if self.processed_df is not None else None,
            'zones_created': len(self.zone_mapper_df) if self.zone_mapper_df is not None else 0,
            'columns_dropped': len(self.cols_to_drop_),
            'columns_skewness_corrected': len(self.skewed_cols_),
            'high_correlation_pairs': len(self.high_corr_df_products) if self.high_corr_df_products is not None else 0
        }

In [3]:
os.chdir("../..")  # subir al nivel raíz del proyecto
!pwd  # verificar ruta actual



/Users/luis.caporal/Documents/Master - TEC/9. MLOps/MNA_MLOps_Equipo12_backup


In [4]:
!dvc pull data/raw/insurance_company_original.csv.dvc


zsh:1: command not found: dvc


In [5]:
os.chdir("notebooks")


file_name = '../data/raw/insurance_company_original.csv'
sociodemographic_cols = [f"SD_{i}" for i in range(1, 44)]
product_cols = [f"PD_{i-44}" for i in range(44, 86)]
cols = sociodemographic_cols + product_cols + ["target"]
df = pd.read_csv(file_name, header=None, names=cols)
df = df.iloc[1:].reset_index(drop=True)


In [6]:
preprocessor = Preprocessor(df)
processed = preprocessor.apply_preprocess(sociodemographic_cols, product_cols)

2025-11-12 19:11:17,176 - preprocessor_logger - INFO - [__init__] - Preprocessor inicializado - Shape original: (5821, 86)
2025-11-12 19:11:17,177 - preprocessor_logger - INFO - [apply_preprocess] - === INICIANDO PIPELINE COMPLETO DE PREPROCESAMIENTO ===
2025-11-12 19:11:17,177 - preprocessor_logger - INFO - [apply_preprocess] - Columnas sociodemográficas: ['SD_1', 'SD_2', 'SD_3', 'SD_4', 'SD_5', 'SD_6', 'SD_7', 'SD_8', 'SD_9', 'SD_10', 'SD_11', 'SD_12', 'SD_13', 'SD_14', 'SD_15', 'SD_16', 'SD_17', 'SD_18', 'SD_19', 'SD_20', 'SD_21', 'SD_22', 'SD_23', 'SD_24', 'SD_25', 'SD_26', 'SD_27', 'SD_28', 'SD_29', 'SD_30', 'SD_31', 'SD_32', 'SD_33', 'SD_34', 'SD_35', 'SD_36', 'SD_37', 'SD_38', 'SD_39', 'SD_40', 'SD_41', 'SD_42', 'SD_43']
2025-11-12 19:11:17,177 - preprocessor_logger - INFO - [apply_preprocess] - Columnas de productos: ['PD_0', 'PD_1', 'PD_2', 'PD_3', 'PD_4', 'PD_5', 'PD_6', 'PD_7', 'PD_8', 'PD_9', 'PD_10', 'PD_11', 'PD_12', 'PD_13', 'PD_14', 'PD_15', 'PD_16', 'PD_17', 'PD_18', '

In [7]:
import mlflow
import mlflow.sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [8]:
target = "target"
X = processed.drop(columns=[target])
y = processed[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Para la regresión logistica
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Configuración de MLFlow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("MNA_MLOps_Model_Comparison")


<Experiment: artifact_location='mlflow-artifacts:/480094317194489582', creation_time=1762995860178, experiment_id='480094317194489582', last_update_time=1762995860178, lifecycle_stage='active', name='MNA_MLOps_Model_Comparison', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [10]:



def train_and_log_model(model, model_name, X_train, X_test, y_train, y_test, params=None):
    with mlflow.start_run(run_name=model_name):
        # Registrar nombre del modelo como parámetro y tag
        mlflow.log_param("model_name", model_name)
        mlflow.set_tag("model_name", model_name)
        
        mlflow.log_param("base_model", model_name)
        mlflow.log_param("model_id", model_name + "_individual")
        
        
        if params:
            mlflow.log_params(params)
        
        # Entrenamiento
        model.fit(X_train, y_train)
        
        # Predicciones
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None
        
        # Métricas
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, zero_division=0),
            "recall": recall_score(y_test, y_pred, zero_division=0),
            "f1_score": f1_score(y_test, y_pred, zero_division=0),
        }
        if y_proba is not None:
            metrics["roc_auc"] = roc_auc_score(y_test, y_proba)
            
        cm_normalized = confusion_matrix(y_test, y_pred)
        labels = sorted(list(set(y_test)))

        plt.figure(figsize=(5, 4))
        sns.heatmap(cm_normalized, annot=True, fmt=".2f", cmap="Blues",
                    xticklabels=labels, yticklabels=labels)
        plt.xlabel("Predicted label")
        plt.ylabel("True label")
        plt.title(f"Matriz de Confusión - {model_name}")
        plt.tight_layout()

        # Guardar imagen localmente
        os.makedirs("./reports/confusion_matrices", exist_ok=True)
        cm_path = f"./reports/confusion_matrices/confusion_matrix_{model_name}.png"
        plt.savefig(cm_path)
        plt.close()

        # Registrar en MLflow
        mlflow.log_artifact(cm_path)

        mlflow.log_metrics(metrics)
        
        # Log del modelo
        # mlflow.sklearn.log_model(model, name=f"model_{model_name}")

        print(f"✅ {model_name} registrado en MLflow con métricas:")
        for k, v in metrics.items():
            print(f"   {k}: {v:.4f}")


# ============================================
# Entrenamiento de modelos
# ============================================

# --- Regresión Logística ---
log_reg_params = {"solver": "liblinear", "random_state": 42}
log_reg = LogisticRegression(**log_reg_params)
train_and_log_model(log_reg, "LogisticRegression", X_train_scaled, X_test_scaled, y_train, y_test, log_reg_params)

# --- Random Forest ---
rf_params = {"n_estimators": 200, "max_depth": 10, "random_state": 42}
rf = RandomForestClassifier(**rf_params)
train_and_log_model(rf, "RandomForest", X_train, X_test, y_train, y_test, rf_params)

# --- XGBoost ---
xgb_params = {
    "n_estimators": 300,
    "max_depth": 5,
    "learning_rate": 0.05,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "random_state": 42,
    "use_label_encoder": False,
    "eval_metric": "logloss",
}
xgb = XGBClassifier(**xgb_params)
train_and_log_model(xgb, "XGBoost", X_train, X_test, y_train, y_test, xgb_params)


✅ LogisticRegression registrado en MLflow con métricas:
   accuracy: 0.9288
   precision: 0.2000
   recall: 0.0862
   f1_score: 0.1205
   roc_auc: 0.6732
🏃 View run LogisticRegression at: http://localhost:5000/#/experiments/480094317194489582/runs/7764519c9bfc4314ac9491120b73b55e
🧪 View experiment at: http://localhost:5000/#/experiments/480094317194489582
✅ RandomForest registrado en MLflow con métricas:
   accuracy: 0.9434
   precision: 0.0000
   recall: 0.0000
   f1_score: 0.0000
   roc_auc: 0.7500
🏃 View run RandomForest at: http://localhost:5000/#/experiments/480094317194489582/runs/3316f0864c244fa98d05dbe527541412
🧪 View experiment at: http://localhost:5000/#/experiments/480094317194489582


/Users/luis.caporal/miniconda3/envs/notebooks/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [19:11:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ XGBoost registrado en MLflow con métricas:
   accuracy: 0.9434
   precision: 0.5000
   recall: 0.0172
   f1_score: 0.0333
   roc_auc: 0.7498
🏃 View run XGBoost at: http://localhost:5000/#/experiments/480094317194489582/runs/f760884e03a54691a5b566e2555017b4
🧪 View experiment at: http://localhost:5000/#/experiments/480094317194489582


In [11]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import logging
from typing import Union, Optional, Dict, Any
import numpy as np
from sklearn.metrics import roc_curve, auc
import os

logger = logging.getLogger('preprocessor_logger')
logger.setLevel(logging.INFO)


class H2OAutoMLTrainer:
    def __init__(self, max_models: int = 20, max_runtime_secs: int = 300,
                 seed: int = 42, balance_classes: bool = True,
                 stopping_metric: str = "AUC"):
        """
        Inicializa el entrenador de H2O AutoML.

        Args:
            max_models: Número máximo de modelos a entrenar
            max_runtime_secs: Tiempo máximo de ejecución en segundos
            seed: Semilla para reproducibilidad
            balance_classes: Balancear clases para datos desbalanceados
            stopping_metric: Métrica para early stopping
        """
        self.max_models = max_models
        self.max_runtime_secs = max_runtime_secs
        self.seed = seed
        self.balance_classes = balance_classes
        self.stopping_metric = stopping_metric

        # Atributos del modelo
        self.aml = None
        self.leader = None
        self.leaderboard = None
        self.h2o_frame = None

        # Estado de H2O
        self.h2o_initialized = False

        logger.info(f"H2OAutoMLTrainer inicializado - max_models: {max_models}, "
                    f"max_runtime_secs: {max_runtime_secs}")

    def initialize_h2o(self) -> None:
        """Inicializa la conexión con H2O."""
        if not self.h2o_initialized:
            try:
                h2o.init()
                self.h2o_initialized = True
                logger.info("H2O inicializado exitosamente")
            except Exception as e:
                logger.error(f"Error inicializando H2O: {e}")
                raise

    def prepare_data(self, df: pd.DataFrame, target_col: str = 'target') -> None:
        """
        Prepara los datos para H2O AutoML.

        Args:
            df: DataFrame con los datos
            target_col: Nombre de la columna target
        """
        logger.info(f"Preparando datos - Shape: {df.shape}, Target: {target_col}")

        if not self.h2o_initialized:
            self.initialize_h2o()

        # Convertir a H2OFrame
        self.h2o_frame = h2o.H2OFrame(df)

        # Convertir target a factor (para clasificación)
        if target_col in self.h2o_frame.columns:
            self.h2o_frame[target_col] = self.h2o_frame[target_col].asfactor()
            logger.info(f"Target '{target_col}' convertido a factor")

        logger.info(f"Datos preparados - H2OFrame shape: {self.h2o_frame.shape}")

    def train(self, target_col: str = 'target',
              training_frame: Optional[h2o.H2OFrame] = None) -> None:
        """
        Entrena el modelo AutoML.

        Args:
            target_col: Columna target
            training_frame: Frame de entrenamiento (opcional, usa self.h2o_frame por defecto)
        """
        logger.info("Iniciando entrenamiento AutoML")

        if training_frame is None:
            if self.h2o_frame is None:
                raise ValueError("No hay datos preparados. Ejecute prepare_data primero.")
            training_frame = self.h2o_frame

        # Configurar AutoML
        self.aml = H2OAutoML(
            max_models=self.max_models,
            seed=self.seed,
            max_runtime_secs=self.max_runtime_secs,
            balance_classes=self.balance_classes,
            stopping_metric=self.stopping_metric
        )

        # Entrenar
        self.aml.train(y=target_col, training_frame=training_frame)

        # Obtener resultados
        self.leader = self.aml.leader
        self.leaderboard = self.aml.leaderboard

        logger.info("Entrenamiento AutoML completado exitosamente")
        self._log_training_summary()

    def _log_training_summary(self) -> None:
        """Registra un resumen del entrenamiento."""
        if self.leaderboard is not None:
            lb_head = self.leaderboard.head()
            logger.info("Leaderboard - Top modelos:")
            for i, (model_id, *metrics) in enumerate(lb_head.as_data_frame().itertuples(index=False)):
                logger.info(f"  {i + 1}. {model_id} - {metrics}")

        if self.leader is not None:
            logger.info(f"Modelo líder: {self.leader.model_id}")

    def predict(self, X_data: Union[pd.DataFrame, h2o.H2OFrame],
                best_threshold: Optional[float] = None,
                auto_threshold: bool = False,
                y_true: Optional[np.ndarray] = None,
                return_h2o_frame: bool = False) -> Union[pd.DataFrame, h2o.H2OFrame]:
        """
        Realiza predicciones con el modelo líder.

        Args:
            X_data: Datos para predecir
            best_threshold: Umbral óptimo para clasificación (opcional)
            auto_threshold: Si calcular automáticamente el threshold que maximiza AUC
            y_true: Valores reales (requerido si auto_threshold=True)
            return_h2o_frame: Si retornar H2OFrame en lugar de DataFrame

        Returns:
            DataFrame o H2OFrame con predicciones
        """
        logger.info("Realizando predicciones")

        if self.leader is None:
            raise ValueError("No hay modelo entrenado. Ejecute train primero.")

        # Validar parámetros para auto_threshold
        if auto_threshold and y_true is None:
            raise ValueError("y_true es requerido cuando auto_threshold=True")

        # Convertir a H2OFrame si es necesario
        if isinstance(X_data, pd.DataFrame):
            h2o_X = h2o.H2OFrame(X_data)
            logger.info(f"DataFrame convertido a H2OFrame - Shape: {h2o_X.shape}")
        else:
            h2o_X = X_data

        # Realizar predicciones
        predictions = self.leader.predict(h2o_X)

        if return_h2o_frame:
            # Combinar features con predicciones
            results_h2o = h2o_X.cbind(predictions)
            logger.info("Predicciones completadas - Retornando H2OFrame")
            return results_h2o
        else:
            # Convertir a DataFrame de pandas
            features_df = h2o_X.as_data_frame().copy()
            probabilities = predictions['p1'].as_data_frame().values.flatten()
            predicted_classes = predictions['predict'].as_data_frame().values.flatten()

            results_df = features_df.copy()
            results_df['predicted_prob'] = probabilities
            results_df['predicted_class'] = predicted_classes

            # Calcular threshold automáticamente si se solicita
            if auto_threshold:
                # Calcular curva ROC
                fpr, tpr, thresholds = roc_curve(y_true, probabilities)
                roc_auc = auc(fpr, tpr)

                # Encontrar threshold que maximiza AUC (punto más alejado de la diagonal)
                optimal_idx = np.argmax(tpr - fpr)
                best_threshold = thresholds[optimal_idx]

                logger.info(f"Threshold automático calculado: {best_threshold:.4f} (AUC: {roc_auc:.4f})")

            # Aplicar threshold si se proporciona o se calcula automáticamente
            if best_threshold is not None:
                logger.info(f"Umbral aplicado: {best_threshold}")

                results_df['prediction'] = ((results_df['predicted_prob'] >= best_threshold) &
                                            (results_df['predicted_class'] == 1)).astype(int)
            else:
                # Si no hay threshold, usar predicted_class directamente
                results_df['prediction'] = results_df['predicted_class']

            logger.info(f"Predicciones completadas - DataFrame shape: {results_df.shape}")
            return results_df

    def predict_with_actuals(self, X_data: Union[pd.DataFrame, h2o.H2OFrame],
                             y_actual: pd.Series,
                             best_threshold: Optional[float] = None) -> pd.DataFrame:
        """
        Realiza predicciones e incluye los valores reales.

        Args:
            X_data: Features para predecir
            y_actual: Valores reales del target
            best_threshold: Umbral óptimo para clasificación

        Returns:
            DataFrame con predicciones y valores reales
        """
        logger.info("Realizando predicciones con valores reales")

        # Obtener predicciones base
        results_df = self.predict(X_data, best_threshold=best_threshold)

        # Agregar valores reales
        results_df['actual'] = y_actual.values

        logger.info(f"Predicciones con valores reales completadas - Shape: {results_df.shape}")
        return results_df

    def get_model_performance(self, test_data: Optional[h2o.H2OFrame] = None,
                              target_col: str = 'target') -> Any:
        """
        Evalúa el performance del modelo líder.

        Args:
            test_data: Datos de test (opcional)
            target_col: Columna target

        Returns:
            Objeto de performance del modelo
        """
        logger.info("Evaluando performance del modelo")

        if self.leader is None:
            raise ValueError("No hay modelo entrenado. Ejecute train primero.")

        if test_data is None:
            if self.h2o_frame is None:
                raise ValueError("No hay datos disponibles para evaluación")
            test_data = self.h2o_frame

        performance = self.leader.model_performance(test_data=test_data)

        # Log de métricas principales
        if hasattr(performance, 'auc'):
            logger.info(f"AUC: {performance.auc():.4f}")

        if hasattr(performance, 'logloss'):
            logger.info(f"LogLoss: {performance.logloss():.4f}")
        
        if hasattr(performance, 'accuracy'):
            acc_list = performance.accuracy()
            if isinstance(acc_list, list) and len(acc_list) > 0:
                acc_value = acc_list[0][1]
                logger.info(f"Accuracy: {acc_value:.4f}")
            else:
                logger.info("Accuracy: N/A")
        
                return performance

    def get_leaderboard(self, as_dataframe: bool = True) -> Union[h2o.H2OFrame, pd.DataFrame]:
        """
        Obtiene el leaderboard de modelos.

        Args:
            as_dataframe: Si retornar como DataFrame de pandas

        Returns:
            Leaderboard de modelos
        """
        if self.leaderboard is None:
            raise ValueError("No hay leaderboard disponible. Ejecute train primero.")

        if as_dataframe:
            return self.leaderboard.as_data_frame()
        else:
            return self.leaderboard

    def save_model(self, model_name: str = "h2o_automl_model", base_path: str = "./") -> str:
        """
        Guarda el modelo en disco con nombre configurable.

        Args:
            model_name: Nombre del modelo (sin extensión)
            base_path: Ruta base donde guardar el modelo

        Returns:
            Ruta completa donde se guardó el modelo
        """
        if self.leader is None:
            raise ValueError("No hay modelo líder para guardar")

        # Crear directorio si no existe
        os.makedirs(base_path, exist_ok=True)

        # Guardar modelo
        model_path = h2o.save_model(
            model=self.leader,
            path=base_path,
            force=True,
            filename=model_name
        )

        logger.info(f"Modelo guardado en: {model_path}")

    def shutdown_h2o(self) -> None:
        """Cierra la conexión con H2O."""
        if self.h2o_initialized:
            h2o.cluster().shutdown()
            self.h2o_initialized = False
            logger.info("H2O shutdown completado")

    def get_training_summary(self) -> Dict[str, Any]:
        """
        Retorna un resumen del entrenamiento.

        Returns:
            Diccionario con información del entrenamiento
        """
        summary = {
            'max_models': self.max_models,
            'max_runtime_secs': self.max_runtime_secs,
            'seed': self.seed,
            'balance_classes': self.balance_classes,
            'stopping_metric': self.stopping_metric,
            'leader_model': self.leader.model_id if self.leader else None,
            'models_trained': len(self.leaderboard) if self.leaderboard else 0,
            'h2o_initialized': self.h2o_initialized
        }

        if self.leaderboard is not None and len(self.leaderboard) > 0:
            lb_df = self.leaderboard.as_data_frame()
            summary['top_model_auc'] = lb_df.iloc[0]['auc'] if 'auc' in lb_df.columns else None

        return summary

    def __enter__(self):
        """Support for context manager."""
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Cleanup on context manager exit."""
        self.shutdown_h2o()

In [12]:
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc
import numpy as np

def plot_interactive_roc(y_true, y_score, model_name="Model"):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)

    # Crear traza principal de la curva ROC
    trace_main = go.Scatter(
        x=fpr, 
        y=tpr,
        mode='lines+markers',
        line=dict(color='orange', width=2),
        hovertemplate=(
            "<b>Threshold:</b> %{customdata[0]:.3f}<br>"
            "<b>TPR (Recall):</b> %{y:.3f}<br>"
            "<b>FPR:</b> %{x:.3f}<br>"
        ),
        name=f"ROC (AUC = {roc_auc:.3f})",
        customdata=np.stack((thresholds,), axis=-1)
    )

    # Línea diagonal (clasificador aleatorio)
    trace_ref = go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(color='navy', dash='dash'),
        showlegend=True,
        name='Random Classifier'
    )

    # Crear figura
    fig = go.Figure(data=[trace_main, trace_ref])

    fig.update_layout(
        title=f"Interactive ROC Curve - {model_name}",
        xaxis_title="False Positive Rate (FPR)",
        yaxis_title="True Positive Rate (TPR)",
        template="plotly_white",
        width=800, height=600,
        hovermode="closest",
        legend=dict(x=0.65, y=0.1, bgcolor='rgba(255,255,255,0.7)')
    )
    
    os.makedirs("./visualization/roc_curves", exist_ok=True)
    roc_path = f"./visualization/roc_curves/cm_{model_name}.html"

    fig.write_html(roc_path)
    mlflow.log_artifact(roc_path)


In [13]:
import os
import mlflow
import mlflow.h2o
import h2o
import logging
from sklearn.model_selection import train_test_split

# ==========================================================
# CONFIGURACIÓN BASE
# ==========================================================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

target = "target"

X = processed.drop(columns=[target])
y = processed[target]

df_train, df_test = train_test_split(processed, test_size=0.2, random_state=42, stratify=processed[target])

# ==========================================================
# CONFIGURAR Y ENTRENAR H2O AutoML
# ==========================================================
with mlflow.start_run(run_name="H2O_AutoML_Master_Run"):

    trainer = H2OAutoMLTrainer(
        max_models=20,
        max_runtime_secs=300,
        seed=42,
        balance_classes=True,
        stopping_metric="AUC"
    )

    # Inicializar y preparar datos
    trainer.prepare_data(df_train, target_col=target)

    # Entrenar modelo
    trainer.train(target_col=target)

    # ======================================================
    # LOGS EN MLFLOW - PARÁMETROS BASE
    # ======================================================
    summary = trainer.get_training_summary()
    mlflow.log_params({
        "max_models": summary["max_models"],
        "max_runtime_secs": summary["max_runtime_secs"],
        "seed": summary["seed"],
        "balance_classes": summary["balance_classes"],
        "stopping_metric": summary["stopping_metric"],
    })

    # ======================================================
    # LEADERBOARD COMPLETO
    # ======================================================
    leaderboard_df = trainer.get_leaderboard(as_dataframe=True)
    os.makedirs("./models", exist_ok=True)
    leaderboard_path = "./models/h2o_leaderboard.csv"
    leaderboard_df.to_csv(leaderboard_path, index=False)
    mlflow.log_artifact(leaderboard_path)

    logger.info("Registrando métricas individuales de cada modelo del leaderboard...")

    # ======================================================
    # REGISTRAR CADA MODELO INDIVIDUAL EN MLFLOW
    # ======================================================

    for idx, row in leaderboard_df.iterrows():
        model_id = row["model_id"]
        model = h2o.get_model(model_id)

        # Calcular performance del modelo sobre test set
        test_h2o = h2o.H2OFrame(df_test)
        performance = model.model_performance(test_h2o)

        # ======== MÉTRICAS BASE ========
        acc_list = performance.accuracy()
        acc_value = acc_list[0][1] if isinstance(acc_list, list) and len(acc_list) > 0 else None
        auc_value = performance.auc() if hasattr(performance, "auc") else None
        logloss_value = performance.logloss() if hasattr(performance, "logloss") else None

        # ======== PREDICCIONES PARA ROC ========
        preds = model.predict(test_h2o).as_data_frame()
        results_df = df_test.copy()
        results_df["predicted_prob"] = preds["p1"].values
        results_df["predicted_class"] = preds["predict"].values
    
        # ======== DETECTAR MODELO BASE ========
        base_model = getattr(model, "algo", None)
        if base_model is None:
            if "StackedEnsemble" in model_id:
                base_model = "StackedEnsemble"
            else:
                base_model = "Unknown"

        # ======== MATRIZ DE CONFUSIÓN ========
        try:
            cm = performance.confusion_matrix()
            cm_df = cm.table.as_data_frame()
            labels = cm_df.iloc[:2, 0].values
            cm_values = cm_df.iloc[:2, 1:3].astype(int).values

            # Extraer métricas derivadas (binarias)
            # Estructura de la matriz:
            #  [[TN, FP],
            #   [FN, TP]]
            if cm_values.shape == (2, 2):
                tn, fp, fn, tp = cm_values.flatten()
                precision = tp / (tp + fp) if (tp + fp) > 0 else 0
                recall = tp / (tp + fn) if (tp + fn) > 0 else 0
                f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            else:
                # Para problemas multiclase, usar promedio macro
                precisions, recalls, f1s = [], [], []
                for i in range(cm_values.shape[0]):
                    tp = cm_values[i, i]
                    fp = np.sum(cm_values[:, i]) - tp
                    fn = np.sum(cm_values[i, :]) - tp
                    precision_i = tp / (tp + fp) if (tp + fp) > 0 else 0
                    recall_i = tp / (tp + fn) if (tp + fn) > 0 else 0
                    f1_i = 2 * precision_i * recall_i / (precision_i + recall_i) if (precision_i + recall_i) > 0 else 0
                    precisions.append(precision_i)
                    recalls.append(recall_i)
                    f1s.append(f1_i)
                precision = np.mean(precisions)
                recall = np.mean(recalls)
                f1_score = np.mean(f1s)

        except Exception as e:
            logger.warning(f"No se pudo obtener la matriz de confusión para {model_id}: {e}")
            continue

        # ======== SUB-RUN EN MLFLOW ========
        with mlflow.start_run(run_name=f"H2O_Model_{model_id}", nested=True):
            mlflow.log_params({
                "model_id": model_id,
                "base_model": base_model.capitalize()
            })
            mlflow.log_metrics({
                "AUC": auc_value,
                "logloss": logloss_value,
                "accuracy": acc_value,
                "precision": precision,
                "recall": recall,
                "f1_score": f1_score
            })
            mlflow.log_param("model_name", model_id)
            logger.info(f"✅ Modelo {model_id} registrado: ")
            mlflow.set_tag("model_name", model_id)

            # ======== GUARDAR MODELO ========
            model_path = h2o.save_model(model=model, path="./models", force=True)
            mlflow.log_artifact(model_path)

            # ======== MATRIZ DE CONFUSIÓN COMO IMAGEN ========
            plt.figure(figsize=(5, 4))
            sns.heatmap(cm_values, annot=True, fmt="d", cmap="Blues",
                        xticklabels=labels, yticklabels=labels)
            plt.xlabel("Predicted")
            plt.ylabel("Actual")
            plt.title(f"Confusion Matrix - {model_id}")
            plt.tight_layout()
            
            os.makedirs("./visualization/confusion_matrices", exist_ok=True)
            cm_path = f"./visualization/confusion_matrices/cm_{model_id}.png"

            plt.savefig(cm_path)
            plt.close()
            
            
            try:
                plot_interactive_roc(results_df["target"], results_df["predicted_prob"], model_name=model_id)
            except Exception as e:
                logger.warning(f"No se pudo generar curva ROC para {model_id}: {e}")

            mlflow.log_artifact(cm_path)

        logger.info(f"✅ Modelo {model_id} registrado: ")

    # ======================================================
    # REGISTRAR EL MODELO LÍDER FINAL
    # ======================================================
    leader_perf = trainer.get_model_performance(target_col=target)
    # mlflow.h2o.log_model(trainer.leader, name="leader_model")

    save_path = trainer.save_model(model_name="h2o_automl_best_model", base_path="./models")

    print("\n✅ Entrenamiento completado con éxito.")
    print(f"Mejor modelo: {trainer.leader.model_id}")




INFO:__main__:H2OAutoMLTrainer inicializado - max_models: 20, max_runtime_secs: 300
INFO:__main__:Preparando datos - Shape: (5121, 1770), Target: target


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "23.0.2" 2025-01-21; OpenJDK Runtime Environment Homebrew (build 23.0.2); OpenJDK 64-Bit Server VM Homebrew (build 23.0.2, mixed mode, sharing)
  Starting server from /Users/luis.caporal/miniconda3/envs/notebooks/lib/python3.11/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/v4/2y3d_dfd7hv8b31yqy3186v40000gp/T/tmp0eo61_hv
  JVM stdout: /var/folders/v4/2y3d_dfd7hv8b31yqy3186v40000gp/T/tmp0eo61_hv/h2o_luis_caporal_started_from_python.out
  JVM stderr: /var/folders/v4/2y3d_dfd7hv8b31yqy3186v40000gp/T/tmp0eo61_hv/h2o_luis_caporal_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Mexico_City
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.8
H2O_cluster_version_age:,1 month and 4 days
H2O_cluster_name:,H2O_from_python_luis_caporal_b1tb7z
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.979 Gb
H2O_cluster_total_cores:,10
H2O_cluster_allowed_cores:,10
H2O_cluster_status:,"locked, healthy"


INFO:__main__:H2O inicializado exitosamente


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


...

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |

/Users/luis.caporal/miniconda3/envs/notebooks/lib/python3.11/site-packages/h2o/frame.py:1983: H2ODependencyWarning:

Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)


INFO:__main__:✅ Modelo DeepLearning_1_AutoML_1_20251112_191136 registrado: 
INFO:__main__:✅ Modelo DeepLearning_1_AutoML_1_20251112_191136 registrado: 
INFO:__main__:Evaluando performance del modelo


██████████████████████████████████████████████| (done) 100%
🏃 View run H2O_Model_DeepLearning_1_AutoML_1_20251112_191136 at: http://localhost:5000/#/experiments/480094317194489582/runs/b48cc24f5bb746e389fdfa9ceec481b7
🧪 View experiment at: http://localhost:5000/#/experiments/480094317194489582


INFO:__main__:AUC: 0.8705
INFO:__main__:LogLoss: 0.1641
INFO:__main__:Accuracy: 0.9467
INFO:__main__:Modelo guardado en: /Users/luis.caporal/Documents/Master - TEC/9. MLOps/MNA_MLOps_Equipo12_backup/notebooks/models/h2o_automl_best_model



✅ Entrenamiento completado con éxito.
Mejor modelo: StackedEnsemble_BestOfFamily_1_AutoML_1_20251112_191136
🏃 View run H2O_AutoML_Master_Run at: http://localhost:5000/#/experiments/480094317194489582/runs/b103db1002fb4f9da187142e131719ef
🧪 View experiment at: http://localhost:5000/#/experiments/480094317194489582
